In [5]:
import csv,fnmatch,os,json
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from tqdm import tqdm
import requests, re
import pickle

country_mapping = {'Australia':'AU',
                   'Austria':'AT',
                   'Brazil':'BR',
                   'Canada':'CA',
                   'Colombia':'CO',
                   'France':'FR',
                   'Guyana':'GY',
                   'Hungary':'HU',
                   'India':'IN',
                   'Indonesia':'ID',
                   'Ireland':'IE',
                   'Jamaica':'JM',
                   'Japan':'JP',
                   'Latvia':'LV',
                   'Mexico':'MX',
                   'Netherlands':'NL',
                   'Nigeria':'NG',
                   'Norway':'NO',
                   'ON':'CA',
                   'Panama':'PA',
                   'Philippines':'PH',
                   'Poland':'PL',
                   'Portugal':'PT',
                   'Slovenia':'SI',
                   'South Africa':'ZA',
                   'South Korea':'KR',
                   'Spain':'ES',
                   'Sweden':'SE',
                   'Switzerland':'CH',
                   'Turkey':'TR',
                   'United Kingdom':'GB',
                   'United States':'US',
                   'Venezuela':'VE'
                  }

def map_country_iso(country):
    if country not in country_mapping.keys():
        return 'PROBLEM WITH : %s' % country
    else:
        return country_mapping.get(country)

regex1 = re.compile("u\'|\'")
searching_for_average = re.compile('Average Monthly Disposable Salary \(After Tax\), Salaries And Financing')
def average_salary_extraction(prices): 
    prices.replace(regex1, '\"')#.apply(lambda price: json.loads(price))
    prices = prices.apply(lambda price: [dictio['average_price'] for dictio in price
                                 if searching_for_average.match(dictio['item_name'])])
    return prices.apply(lambda price: price[0])

In [8]:
#Loading data

data_folder = '/home/thebatou/Documents/Uni_Potsdam/Competitive_Data_Analysis/'
file_name = 'project_data_train_v2.csv'

try:
    print 'Trying to load bitbond data ...'
    bitbond_data = pd.read_csv(os.path.join(data_folder, file_name), index_col=0)
    bitbond_data.loc[835,'region'] = 'NA'
    bitbond_data.loc[1232,'region'] = 'NA'
    bitbond_data.loc[1839,'region'] = 'NA'
    bitbond_data.loc[1993,'region'] = 'NA'
    bitbond_data = bitbond_data.replace(to_replace=np.nan, value=-1)
    print 'Done loading bitbond data! {} data were loaded.'.format(bitbond_data.shape[0])
    print ("Trying to load numbeo data about city ...")
    print ("Trying to load numbeo data about country ...")
    city_prices_numbeo = pd.read_csv('/home/thebatou/Documents/Uni_Potsdam/Competitive_Data_Analysis/numbeo_city_prices.csv', index_col=0)
    country_prices_numbeo = pd.read_csv('/home/thebatou/Documents/Uni_Potsdam/Competitive_Data_Analysis/numbeo_country_prices.csv', index_col=0)
    print "Done loading city numbeo data! {} data were loaded.".format(city_prices_numbeo.shape[0])
    print "Done loading country numbeo data! {} data were loaded.".format(country_prices_numbeo.shape[0])
    # DROP USELESS COLUMN
    city_prices_numbeo.drop('currency',axis=1,inplace=True)
    city_prices_numbeo.drop('contributors',axis=1,inplace=True)
    city_prices_numbeo.drop('monthLastUpdate',axis=1,inplace=True)
    city_prices_numbeo.drop('yearLastUpdate',axis=1,inplace=True)
    city_prices_numbeo = city_prices_numbeo.rename(columns={'name':'location'})

    country_prices_numbeo.drop('currency',axis=1,inplace=True)
    country_prices_numbeo.drop('contributors',axis=1,inplace=True)
    country_prices_numbeo.drop('monthLastUpdate',axis=1,inplace=True)
    country_prices_numbeo.drop('yearLastUpdate',axis=1,inplace=True)
    country_prices_numbeo = country_prices_numbeo.rename(columns={'name':'location'})
except IOError as e:
    print "I/O error: {0}".format(e)
    print "Downloading data from numbeo website ..."
    quadruple_id_lat_long_region = bitbond_data.groupby(by=('loan_identifier','address_lat','address_lng','region')).indices.keys()
    all_city_prices_requests = []
    all_country_prices_requests = []
    missing_prices_location = []
    for address in tqdm(quadruple_id_lat_long_region):
        # CITY REQUEST
        url = "http://www.numbeo.com/api/city_prices?api_key=w624aixu3sour8&query="+str(address[1])+','+str(address[2])
        response = json.loads(requests.get(url).text)
        # CHECK CITY REQUEST RESPONSE
        if 'error' in response.keys():
            # COUNTRY REQUEST BECAUSE CITY REQUEST FAILED
            url = "http://www.numbeo.com/api/country_prices?api_key=w624aixu3sour8&country="+address[3]
            response = json.loads(requests.get(url).text)
            # CHECK COUNTRY RESPONSE
            if 'error' in response.keys():
                print "ERROR CITY-COUNTRY"
                print address
                missing_prices_location.append(address)
                continue
            else:
                # CHECK ITEM OF LIST
                if 'prices' in response.keys() and isinstance(response.get('prices'), list):
                    boolean = False
                    for dico in response.get('prices'):
                        if 105 in dico.values():
                            response['loan_identifier'] = address[0]
                            response['region'] = address[3]
                            all_country_prices_requests.append(response)
                            boolean = True
                            break
                    if not boolean:
                        print "ERROR CITY + NOT ALL PRICE PRESENT IN COUNTRY REQUEST"
                        print address
                        missing_prices_location.append(address)
                        continue
        else:
            if 'prices' in response.keys() and isinstance(response.get('prices'), list):
                boolean = False
                for dico in response.get('prices'):
                    if 105 in dico.values():
                        response['loan_identifier'] = address[0]
                        response['address_lat'] = address[1]
                        response['address_lng'] = address[2]
                        all_city_prices_requests.append(response)
                        boolean = True
                        break
                if not boolean:
                    country = response['name'].split(', ')
                    if len(country)==3 and country[2]=="United States":
                        country = country[2]
                    else:
                        country = country[1]
                    country = map_country_iso(country)
                    url = "http://www.numbeo.com/api/country_prices?api_key=w624aixu3sour8&country="+country
                    response = json.loads(requests.get(url).text)
                    if 'error' in response.keys() or 'prices' not in response.keys():
                        print "ERROR COUNTRY - CITY NOT FOUND"
                        print country
                        print address
                        missing_prices_location.append(address)
                        continue
    
    with open('/home/thebatou/Documents/Uni_Potsdam/Competitive_Data_Analysis/missings.pickle', "wb") as f:
        pickle.dump(missing_prices_location, f)
    #missing_prices_location.to_csv(path_or_buf='/home/thebatou/Documents/Uni_Potsdam/Competitive_Data_Analysis/missings.csv', encoding='utf-8')
    city_prices_numbeo = pd.DataFrame(all_city_prices_requests)
    # Extraction of average salary per location
    city_prices_numbeo['prices'] = average_salary_extraction(city_prices_numbeo['prices'])
    city_prices_numbeo.to_csv(path_or_buf='/home/thebatou/Documents/Uni_Potsdam/Competitive_Data_Analysis/numbeo_city_prices.csv', encoding='utf-8')
    
    country_prices_numbeo = pd.DataFrame(all_country_prices_requests)
    # Extraction of average salary per location
    country_prices_numbeo['prices'] = average_salary_extraction(country_prices_numbeo['prices'])
    country_prices_numbeo.to_csv(path_or_buf='/home/thebatou/Documents/Uni_Potsdam/Competitive_Data_Analysis/numbeo_country_prices.csv', encoding='utf-8')
    
    city_prices_numbeo.drop('currency',axis=1,inplace=True)
    city_prices_numbeo.drop('contributors',axis=1,inplace=True)
    city_prices_numbeo.drop('monthLastUpdate',axis=1,inplace=True)
    city_prices_numbeo.drop('yearLastUpdate',axis=1,inplace=True)
    city_prices_numbeo = city_prices_numbeo.rename(columns={'name':'location'})

    country_prices_numbeo.drop('currency',axis=1,inplace=True)
    country_prices_numbeo.drop('contributors',axis=1,inplace=True)
    country_prices_numbeo.drop('monthLastUpdate',axis=1,inplace=True)
    country_prices_numbeo.drop('yearLastUpdate',axis=1,inplace=True)
    country_prices_numbeo = country_prices_numbeo.rename(columns={'name':'location'})
    
print "Done downloading from numbeo website!"
print 'Bitbond data shape : {}'.format(bitbond_data.shape)
print 'City numbeo data has shape : {}'.format(city_prices_numbeo.shape)
print 'Country numbeo data has shape : {}'.format(country_prices_numbeo.shape)
print '\nFirst element of city numbeo data :'
print city_prices_numbeo.ix[0]
print '\nFirst element of country numbeo data :'
print country_prices_numbeo.ix[2]

Loading bitbond data ...
Done loading bitbond data! 2177 data were loaded.
Trying to load numbeo data about city ...
Trying to load numbeo data about country ...
Done loading city numbeo data! 1323 data were loaded.
Done loading country numbeo data! 422 data were loaded.
Done downloading from numbeo website!
Bitbond data shape : (2177, 28)
City numbeo data has shape : (1323, 5)
Country numbeo data has shape : (422, 4)

First element of city numbeo data :
address_lat                  -7.238452
address_lng                    112.735
loan_identifier             1VSAV13ZD8
location           Surabaya, Indonesia
prices                         3926000
Name: 0, dtype: object

First element of country numbeo data :
loan_identifier    2AAFMY494Z
location               Brazil
prices               1701.551
region                     BR
Name: 2, dtype: object


In [9]:
print len(bitbond_data.groupby(by=('address_lat','address_lng')).indices.keys())
print len(bitbond_data['region'])
print np.unique(bitbond_data['region'])

446
2177
['AR' 'AT' 'AU' 'BD' 'BE' 'BG' 'BR' 'CA' 'CH' 'CI' 'CL' 'CN' 'CO' 'CZ' 'DE'
 'DK' 'DO' 'DZ' 'EC' 'EE' 'EG' 'ES' 'FI' 'FR' 'GB' 'GH' 'GR' 'GY' 'HR' 'HU'
 'ID' 'IE' 'IL' 'IN' 'IR' 'IS' 'IT' 'JM' 'JP' 'KE' 'KR' 'LB' 'LK' 'LT' 'LU'
 'LV' 'MA' 'MK' 'MU' 'MX' 'NA' 'NG' 'NL' 'NO' 'NZ' 'PA' 'PE' 'PH' 'PK' 'PL'
 'PT' 'RO' 'RS' 'RU' 'SE' 'SG' 'SI' 'SK' 'SR' 'SY' 'TH' 'TR' 'TZ' 'UA' 'US'
 'VE' 'VN' 'ZA' 'ZW']


In [10]:
regions = np.unique(bitbond_data.region)
regions_prices_requests = []
missing_prices_regions = []
for region in tqdm(regions):
    url = "http://www.numbeo.com/api/country_prices?api_key=w624aixu3sour8&country="+region
    response = json.loads(requests.get(url).text)
    if 'error' in response.keys():
        missing_prices_regions.append(region)
    else:
        if 'prices' in response.keys() and isinstance(response.get('prices'), list):
            boolean = False
            for dico in response.get('prices'):
                if 105 in dico.values():
                    response['region'] = region
                    regions_prices_requests.append(response)
                    boolean = True
                    break
            if not boolean:
                print "ERROR CITY + NOT ALL PRICE PRESENT IN COUNTRY REQUEST"
                print region
                missing_prices_regions.append(region)
                continue

regions_prices_numbeo = pd.DataFrame(regions_prices_requests)
regions_prices_numbeo['prices'] = average_salary_extraction(regions_prices_numbeo['prices'])
regions_prices_numbeo.drop('currency',axis=1,inplace=True)
regions_prices_numbeo.drop('contributors',axis=1,inplace=True)
regions_prices_numbeo.drop('monthLastUpdate',axis=1,inplace=True)
regions_prices_numbeo.drop('yearLastUpdate',axis=1,inplace=True)
regions_prices_numbeo = regions_prices_numbeo.rename(columns={'name':'location'})

In [11]:
#print missing_prices_location
print missing_prices_regions
regions_prices_numbeo
#missing = pd.DataFrame(missing_prices_address,columns=['loan_identifier','address_lat','address_lng'])
#missing_data = pd.merge(bitbond_data, missing, how='inner', on=['loan_identifier','address_lat', 'address_lng'])

[]


,location,prices,region
0,Argentina,1077.873625,AR
1,Austria,1809.835758,AT
2,Australia,4257.531723,AU
3,Bangladesh,25214.880952,BD
4,Belgium,1848.918875,BE
5,Bulgaria,846.968508,BG
6,Brazil,1745.909005,BR
7,Canada,2997.538016,CA
8,Switzerland,5598.811004,CH
9,Ivory Coast,300000.000000,CI


In [14]:
print bitbond_data.columns
print bitbond_data.ix[10]
print regions_prices_numbeo.ix[1]
print city_prices_numbeo.ix[1]
print country_prices_numbeo.ix[1]
#numbeo_data[numbeo_data['cpi_index']!=np.nan]
#bitbond_data[bitbond_data['region']=='tamere']

Index([u'loan_identifier', u'nominal_interest_rate', u'term', u'purpose',
       u'project_description', u'borrower_identifier', u'borrower_rating',
       u'employment', u'region', u'facebook', u'twitter', u'paypal', u'ebay',
       u'linkedin', u'published_at', u'funded_at', u'status',
       u'amount_requested', u'amount_funded', u'issuer_id', u'fraudulent',
       u'address_lat', u'address_lng', u'net_income_cents', u'currency',
       u'rates_count', u'rates_paid', u'base_currency'],
      dtype='object')
loan_identifier                                                  EHDT531SX
nominal_interest_rate                                                 3332
term                                                         term_6_months
purpose                                                    working_capital
project_description      Hey everybody, I want to open a fitness center...
borrower_identifier                                              EH91S31SV
borrower_rating                   

In [15]:
bitbond_numbeo_data = pd.merge(bitbond_data, regions_prices_numbeo, on=['region'], how='left')
bitbond_numbeo_data_2 = pd.merge(bitbond_numbeo_data, country_prices_numbeo, on=['loan_identifier','region','prices', 'location'], how='left')
data = pd.merge(bitbond_numbeo_data_2, city_prices_numbeo, on=['loan_identifier','address_lat', 'address_lng', 'prices', 'location'], how='left')
data.to_csv(path_or_buf='/home/thebatou/Documents/Uni_Potsdam/Competitive_Data_Analysis/data_num_bit.csv', encoding='utf-8')
#bitbond_numbeo_data.groupby(by='prices').indices.values()
print np.unique(data.region)
print np.unique(data.prices)
data.columns

['AR' 'AT' 'AU' 'BD' 'BE' 'BG' 'BR' 'CA' 'CH' 'CI' 'CL' 'CN' 'CO' 'CZ' 'DE'
 'DK' 'DO' 'DZ' 'EC' 'EE' 'EG' 'ES' 'FI' 'FR' 'GB' 'GH' 'GR' 'GY' 'HR' 'HU'
 'ID' 'IE' 'IL' 'IN' 'IR' 'IS' 'IT' 'JM' 'JP' 'KE' 'KR' 'LB' 'LK' 'LT' 'LU'
 'LV' 'MA' 'MK' 'MU' 'MX' 'NA' 'NG' 'NL' 'NO' 'NZ' 'PA' 'PE' 'PH' 'PK' 'PL'
 'PT' 'RO' 'RS' 'RU' 'SE' 'SG' 'SI' 'SK' 'SR' 'SY' 'TH' 'TR' 'TZ' 'UA' 'US'
 'VE' 'VN' 'ZA' 'ZW']
[  1.79500000e+02   1.80146133e+02   1.95827000e+02   2.37559506e+02
   3.70240077e+02   4.40909091e+02   4.71005736e+02   4.78205054e+02
   5.54153945e+02   5.78385480e+02   5.98452989e+02   6.68307692e+02
   6.91932500e+02   7.02776687e+02   7.87125000e+02   7.98813822e+02
   8.27490180e+02   8.46968508e+02   9.76948276e+02   1.07787362e+03
   1.13200820e+03   1.28023247e+03   1.44783333e+03   1.54871583e+03
   1.67123445e+03   1.74590901e+03   1.80983576e+03   1.84891888e+03
   1.86097083e+03   1.94732176e+03   1.99541960e+03   2.14516431e+03
   2.18086667e+03   2.21091543e+03   2.3503332

Index([      u'loan_identifier', u'nominal_interest_rate',
                        u'term',               u'purpose',
         u'project_description',   u'borrower_identifier',
             u'borrower_rating',            u'employment',
                      u'region',              u'facebook',
                     u'twitter',                u'paypal',
                        u'ebay',              u'linkedin',
                u'published_at',             u'funded_at',
                      u'status',      u'amount_requested',
               u'amount_funded',             u'issuer_id',
                  u'fraudulent',           u'address_lat',
                 u'address_lng',      u'net_income_cents',
                    u'currency',           u'rates_count',
                  u'rates_paid',         u'base_currency',
                    u'location',                u'prices'],
      dtype='object')

In [17]:
#country_prices_numbeo[country_prices_numbeo['location']=='Namibia']
data[data['location']=='Namibia']
#data = data.copy(deep=True)
#data.loc[835,'region'] = 'NA'
#data.loc[1232,'region'] = 'NA'
#data.loc[1839,'region'] = 'NA'
#data.loc[1993,'region'] = 'NA'
data.to_csv(path_or_buf='/home/thebatou/Documents/Uni_Potsdam/Competitive_Data_Analysis/final_data.csv', encoding='utf-8')

In [ ]:
status_terms = {}
for status in np.unique(bitbond_data.status):
    for terms in np.unique(bitbond_data.term):
        status_terms_data = bitbond_data[(bitbond_data['status'] == status) & (bitbond_data.get('term') == terms)]
        status_terms[status+' & '+terms]=float(status_terms_data.size)

In [10]:
print 'Nominal interest rate : {}'.format(np.unique(bitbond_data.nominal_interest_rate))
print 'Term : {}'.format(np.unique(bitbond_data.term))
print 'Purpose : {}'.format(np.unique(bitbond_data.purpose))
print 'Borrower rating : {}'.format(np.unique(bitbond_data.borrower_rating))
print 'Employment : {}'.format(np.unique(bitbond_data.employment))
print 'Region : {}'.format(np.unique(bitbond_data.region))
print 'Facebook : {}'.format(np.unique(bitbond_data.facebook))
print 'Status : {}'.format(np.unique(bitbond_data.status))
print 'Fraudulent : {}'.format(np.unique(bitbond_data.fraudulent))
print 'Currency : {}'.format(np.unique(bitbond_data.currency))
print 'Rates count : {}'.format(np.unique(bitbond_data.rates_count))
print 'Rates paid : {}'.format(np.unique(bitbond_data.rates_paid))
print 'Base currency : {}'.format(np.unique(bitbond_data.base_currency))

Nominal interest rate : [ 766  770  788  844  888 1029 1069 1115 1166 1235 1380 1453 1465 1583 1633
 1806 1947 1994 2071 2108 2253 2457 2458 2463 2564 2792 2836 3319 3332 4048]
Term : ['term_12_months' 'term_36_months' 'term_60_months' 'term_6_months'
 'term_6_weeks']
Purpose : ['consumption' 'education' 'investment' 'other' 'refinancing' 'renovation'
 'working_capital']
Borrower rating : ['A' 'B' 'C' 'D' 'E' 'F']
Employment : ['retired' 'salaried' 'self_employed' 'studying' 'unemployed']
Region : ['AR' 'AT' 'AU' 'BD' 'BE' 'BG' 'BR' 'CA' 'CH' 'CI' 'CL' 'CN' 'CO' 'CZ' 'DE'
 'DK' 'DO' 'DZ' 'EC' 'EE' 'EG' 'ES' 'FI' 'FR' 'GB' 'GH' 'GR' 'GY' 'HR' 'HU'
 'ID' 'IE' 'IL' 'IN' 'IR' 'IS' 'IT' 'JM' 'JP' 'KE' 'KR' 'LB' 'LK' 'LT' 'LU'
 'LV' 'MA' 'MK' 'MU' 'MX' 'NA' 'NG' 'NL' 'NO' 'NZ' 'PA' 'PE' 'PH' 'PK' 'PL'
 'PT' 'RO' 'RS' 'RU' 'SE' 'SG' 'SI' 'SK' 'SR' 'SY' 'TH' 'TR' 'TZ' 'UA' 'US'
 'VE' 'VN' 'ZA' 'ZW']
Facebook : ['connected' 'none']
Status : ['canceled' 'charged_off' 'current' 'defaulted' 'expir

In [11]:
print 'Nominal interest rate : {}'.format(np.unique(data.nominal_interest_rate))
print 'Term : {}'.format(np.unique(data.term))
print 'Purpose : {}'.format(np.unique(data.purpose))
print 'Borrower rating : {}'.format(np.unique(data.borrower_rating))
print 'Employment : {}'.format(np.unique(data.employment))
print 'Region : {}'.format(np.unique(data.region))
print 'Facebook : {}'.format(np.unique(data.facebook))
print 'Ebay : {}'.format(np.unique(data.ebay))
print 'Status : {}'.format(np.unique(data.status))
print 'Fraudulent : {}'.format(np.unique(data.fraudulent))
print 'Currency : {}'.format(np.unique(data.currency))
print 'Rates count : {}'.format(np.unique(data.rates_count))
print 'Rates paid : {}'.format(np.unique(data.rates_paid))
print 'Base currency : {}'.format(np.unique(data.base_currency))
print 'Location : {}'.format(np.unique(data.location))
print 'Amount requested : {}'.format(np.unique(data.amount_requested))
print 'Amount funded : {}'.format(np.unique(data.amount_funded))

Nominal interest rate : [ 766  770  788  844  888 1029 1069 1115 1166 1235 1380 1453 1465 1583 1633
 1806 1947 1994 2071 2108 2253 2457 2458 2463 2564 2792 2836 3319 3332 4048]
Term : ['term_12_months' 'term_36_months' 'term_60_months' 'term_6_months'
 'term_6_weeks']
Purpose : ['consumption' 'education' 'investment' 'other' 'refinancing' 'renovation'
 'working_capital']
Borrower rating : ['A' 'B' 'C' 'D' 'E' 'F']
Employment : ['retired' 'salaried' 'self_employed' 'studying' 'unemployed']
Region : ['AR' 'AT' 'AU' 'BD' 'BE' 'BG' 'BR' 'CA' 'CH' 'CI' 'CL' 'CN' 'CO' 'CZ' 'DE'
 'DK' 'DO' 'DZ' 'EC' 'EE' 'EG' 'ES' 'FI' 'FR' 'GB' 'GH' 'GR' 'GY' 'HR' 'HU'
 'ID' 'IE' 'IL' 'IN' 'IR' 'IS' 'IT' 'JM' 'JP' 'KE' 'KR' 'LB' 'LK' 'LT' 'LU'
 'LV' 'MA' 'MK' 'MU' 'MX' 'NA' 'NG' 'NL' 'NO' 'NZ' 'PA' 'PE' 'PH' 'PK' 'PL'
 'PT' 'RO' 'RS' 'RU' 'SE' 'SG' 'SI' 'SK' 'SR' 'SY' 'TH' 'TR' 'TZ' 'UA' 'US'
 'VE' 'VN' 'ZA' 'ZW']
Facebook : ['connected' 'none']
Ebay : ['connected' 'none']
Status : ['canceled' 'charged_off' 

In [13]:
data['pourcentage_average_salary'] = data['net_income_cents'] / data['prices']
print np.mean(data['pourcentage_average_salary'])
print np.min(data['pourcentage_average_salary'])
print np.max(data['pourcentage_average_salary'])
print np.std(data['pourcentage_average_salary'])

37910.5991989
8.8270341723e-05
5240603.34584
433563.166493


In [12]:
type(data.amount_funded.ix[1])

numpy.int64

In [ ]:
from sklearn.cross_validation import StratifiedKFold,cross_val_score
from sklearn.linear_model import SGDClassifier
from scipy import interp
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer

y = data['status'] != 'fully_paid'
v = DictVectorizer()
D = data.loc[:,['amount_funded','amount_requested']].T.to_dict().values()
#.to_dict()#,'address_lat','address_lng' , 'net_income_cents'
                #'nominal_interest_rate', 'prices', 'rates_count', 'rates_paid']]
X = v.fit_transform(D)

random_state = np.random.RandomState(0)

cv = StratifiedKFold(y, n_folds=6)
clf = svm.SVC(kernel='linear', probability=True,
                     random_state=random_state)

mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
all_tpr = []
scores = []

plt.figure(figsize=(20,10))

for i, (train, test) in tqdm(enumerate(cv)):
    clf.fit(X[train], y[train])
    scores.append(clf.score(X[test], y[test]))
    y_scores = clf.decision_function(X[test])
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y[test], y_scores)
    mean_tpr += interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=1, label='ROC fold %d (area = %0.2f)' % (i, roc_auc))
    # Now predict the value of the digit on the second half:
    expected = y[test]
    predicted = clf.predict(X[test])

    print("Classification report for classifier %s:\n%s\n"
          % (clf, metrics.classification_report(expected, predicted)))
    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')

mean_tpr /= len(cv)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, 'k--',
         label='Mean ROC (area = %0.2f)' % mean_auc, lw=2)

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()